In [1]:
!git clone https://github.com/Tanveer2719/NIDS_Coursework.git

Cloning into 'NIDS_Coursework'...
remote: Enumerating objects: 141, done.
remote: Counting objects: 100% (141/141), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 141 (delta 54), reused 133 (delta 46), pack-reused 0 (from 0)
Receiving objects: 100% (141/141), 168.67 KiB | 3.37 MiB/s, done.
Resolving deltas: 100% (54/54), done.


In [2]:
%cd /kaggle/working/NIDS_Coursework/My_Code
!ls

/kaggle/working/NIDS_Coursework/My_Code
classification.py    main.ipynb     record_level_embedding.py
featureSelection.py  preprocess.py  transformer.py


In [3]:
%rm -r __pycache__

rm: cannot remove '__pycache__': No such file or directory


In [4]:
!git pull origin main

From https://github.com/Tanveer2719/NIDS_Coursework
 * branch            main       -> FETCH_HEAD
Already up to date.


In [5]:
import pandas as pd
import numpy as np

path = "/kaggle/input/unsw-nb-15/UNSW-NB15_combined.csv"

df = pd.read_csv(path)

print(df.shape)
df = df.drop(columns=['id', 'label'])
print(df.shape)

print(f"🧾 Unique class labels: {df['attack_cat'].unique()}")
df['label'] = df['attack_cat'].apply(lambda x: 0 if x == 'Normal' else 1)

print(f'Unique values for label {df["label"].unique()}')

df = df.drop(columns=["attack_cat"])
print(f'new shape{df.shape}')

print(df.dtypes)


(257673, 45)
(257673, 43)
🧾 Unique class labels: ['Normal' 'Backdoor' 'Analysis' 'Fuzzers' 'Shellcode' 'Reconnaissance'
 'Exploits' 'DoS' 'Worms' 'Generic']
Unique values for label [0 1]
new shape(257673, 43)
dur                  float64
proto                 object
service               object
state                 object
spkts                  int64
dpkts                  int64
sbytes                 int64
dbytes                 int64
rate                 float64
sttl                   int64
dttl                   int64
sload                float64
dload                float64
sloss                  int64
dloss                  int64
sinpkt               float64
dinpkt               float64
sjit                 float64
djit                 float64
swin                   int64
stcpb                  int64
dtcpb                  int64
dwin                   int64
tcprtt               float64
synack               float64
ackdat               float64
smean                  int64
dmean   

In [6]:
# perform preprocessing

from preprocess import Preprocess

# separate the features and labels so that the labesl are not encoded
labels = df['label']
features = df.drop(columns=['label'])

pp = Preprocess()
processed_features = pp.fit_transform_df_auto(df = features,n_categorical_levels=32, expected_categorical_format='onehot')

print(processed_features.shape)
processed_features.head()

Encoding the 32 levels for proto
Encoding the 13 levels for service
Encoding the 11 levels for state
(257673, 95)


,dur,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,...,state_2,state_3,state_4,state_5,state_6,state_7,state_8,state_9,state_10,state_11
0,0.027889,0.193225,0.172921,0.331292,0.312312,0.312595,0.997874,1.000000,0.424562,0.534539,...,False,False,False,False,False,False,False,False,False,False
1,0.121803,0.284598,0.393619,0.398471,0.645181,0.316704,0.747160,0.998579,0.401347,0.775718,...,False,False,False,False,False,False,False,False,False,False
2,0.234590,0.224248,0.304405,0.353884,0.574953,0.196832,0.747160,0.998579,0.326962,0.650937,...,False,False,False,False,False,False,False,False,False,False
3,0.239956,0.267974,0.275582,0.388675,0.402879,0.194440,0.747160,0.998579,0.351625,0.479722,...,False,False,False,False,False,False,False,False,False,False
4,0.090294,0.248312,0.209072,0.378428,0.339064,0.256038,0.999294,0.998579,0.402218,0.489854,...,False,False,False,False,False,False,False,False,False,False


In [7]:
# concat the features and the labels
processed_full_df = pd.concat([processed_features, labels], axis=1)
processed_full_df.head()

,dur,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,...,state_3,state_4,state_5,state_6,state_7,state_8,state_9,state_10,state_11,label
0,0.027889,0.193225,0.172921,0.331292,0.312312,0.312595,0.997874,1.000000,0.424562,0.534539,...,False,False,False,False,False,False,False,False,False,0
1,0.121803,0.284598,0.393619,0.398471,0.645181,0.316704,0.747160,0.998579,0.401347,0.775718,...,False,False,False,False,False,False,False,False,False,0
2,0.234590,0.224248,0.304405,0.353884,0.574953,0.196832,0.747160,0.998579,0.326962,0.650937,...,False,False,False,False,False,False,False,False,False,0
3,0.239956,0.267974,0.275582,0.388675,0.402879,0.194440,0.747160,0.998579,0.351625,0.479722,...,False,False,False,False,False,False,False,False,False,0
4,0.090294,0.248312,0.209072,0.378428,0.339064,0.256038,0.999294,0.998579,0.402218,0.489854,...,False,False,False,False,False,False,False,False,False,0


In [8]:
from featureSelection import AutoencoderFeatureSelector

selector = AutoencoderFeatureSelector(encoding_dim=40)

# To get compressed features (autoencoder bottleneck layer)
# encoded_feature_names, X_encoded, y = selector.select_features(processed_full_df, target_column='label')

# To get top 50 original features based on encoder weights
top_feats, X_top, y = selector.select_features(processed_full_df, target_column='label', top_k_features=50)

top_feats

2025-07-19 04:09:02.496959: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752898142.683279      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752898142.738934      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered



🧹 Input features shape: (257673, 95), Target shape: (257673,)
🚫 Skipping normalization (already preprocessed)...
🔧 Building autoencoder (input_dim = 95, encoding_dim = 40)...


I0000 00:00:1752898157.182304      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1752898157.183028      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 95)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        12,288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoded_layer (Dense)           │ (None, 40)             │         5,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │         5,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 95)             │        12,255 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

...

I0000 00:00:1752898160.778934     116 service.cc:148] XLA service 0x79fb0c012b60 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1752898160.779896     116 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1752898160.779915     116 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1752898161.025409     116 cuda_dnn.cc:529] Loaded cuDNN version 90300


  77/3221 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 0.0586  

I0000 00:00:1752898161.928631     116 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3221/3221 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - loss: 0.0056 - val_loss: 2.8058e-04
Epoch 2/100
3221/3221 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - loss: 2.3537e-04 - val_loss: 1.4984e-04
Epoch 3/100
3221/3221 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - loss: 1.3131e-04 - val_loss: 9.9560e-05
Epoch 4/100
3221/3221 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - loss: 8.7315e-05 - val_loss: 7.3194e-05
Epoch 5/100
3221/3221 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - loss: 6.2979e-05 - val_loss: 5.2789e-05
Epoch 6/100
3221/3221 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - loss: 4.8811e-05 - val_loss: 4.1822e-05
Epoch 7/100
3221/3221 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - loss: 4.0923e-05 - val_loss: 4.2618e-05
Epoch 8/100
3221/3221 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - loss: 3.7603e-05 - val_loss: 3.5709e-05
Epoch 9/100
3221/3221 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - loss: 3.4805e-05 - val_loss: 3.5746e-05
Epoch 10/100
3221/3221 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - loss: 3.0791e-05 - val_loss: 3.5584e-05
Epoch 11/100
3221/3221 ━━━━━━━━━━━━━━━━━━━━ 

['proto_21',
 'proto_2',
 'proto_29',
 'proto_28',
 'proto_31',
 'proto_26',
 'proto_32',
 'proto_17',
 'proto_23',
 'service_2',
 'proto_14',
 'proto_16',
 'service_12',
 'proto_20',
 'proto_18',
 'proto_24',
 'state_6',
 'proto_15',
 'service_11',
 'proto_11',
 'proto_9',
 'proto_10',
 'service_13',
 'proto_7',
 'proto_30',
 'state_5',
 'service_9',
 'service_7',
 'is_sm_ips_ports',
 'proto_27',
 'proto_22',
 'proto_12',
 'proto_13',
 'sloss',
 'proto_19',
 'ackdat',
 'service_8',
 'dpkts',
 'dwin',
 'spkts',
 'synack',
 'service_5',
 'proto_6',
 'proto_4',
 'dloss',
 'is_ftp_login',
 'dttl',
 'ct_flw_http_mthd',
 'ct_state_ttl',
 'sinpkt']

In [9]:
columns_to_keep = top_feats

In [10]:
df_selected = processed_full_df[columns_to_keep].copy()
df_selected.shape

(257673, 50)

In [11]:
df_selected.dtypes

proto_21               bool
proto_2                bool
proto_29               bool
proto_28               bool
proto_31               bool
proto_26               bool
proto_32               bool
proto_17               bool
proto_23               bool
service_2              bool
proto_14               bool
proto_16               bool
service_12             bool
proto_20               bool
proto_18               bool
proto_24               bool
state_6                bool
proto_15               bool
service_11             bool
proto_11               bool
proto_9                bool
proto_10               bool
service_13             bool
proto_7                bool
proto_30               bool
state_5                bool
service_9              bool
service_7              bool
is_sm_ips_ports     float64
proto_27               bool
proto_22               bool
proto_12               bool
proto_13               bool
sloss               float64
proto_19               bool
ackdat              

In [12]:
numerical_columns = [col for col in columns_to_keep 
                     if not (col.startswith("proto_") or 
                             col.startswith("service_") or 
                             col.startswith("state_"))]


In [13]:
from record_level_embedding import RecordLevelEmbedder

embedder = RecordLevelEmbedder(selected_df = df_selected, numerical_columns = numerical_columns, embed_dimension= 64)

embedded_df = embedder.transform_to_df()

embedded_df.head()

[INFO] Initializing RecordLevelEmbedder...
[INFO] Detecting categorical blocks...
[INFO] Detected 3 categorical blocks:
   - proto: 27 columns
   - service: 8 columns
   - state: 2 columns
[INFO] Embedder initialized with embedding dimension 64.
[INFO] Building Keras embedding model...
   - Adding numerical input: is_sm_ips_ports
   - Adding numerical input: sloss
   - Adding numerical input: ackdat
   - Adding numerical input: dpkts
   - Adding numerical input: dwin
   - Adding numerical input: spkts
   - Adding numerical input: synack
   - Adding numerical input: dloss
   - Adding numerical input: is_ftp_login
   - Adding numerical input: dttl
   - Adding numerical input: ct_flw_http_mthd
   - Adding numerical input: ct_state_ttl
   - Adding numerical input: sinpkt
   - Adding categorical block input: proto (27 columns)
   - Adding categorical block input: service (8 columns)
   - Adding categorical block input: state (2 columns)
[INFO] Concatenating 16 inputs and applying Dense proj

,embed_0,embed_1,embed_2,embed_3,embed_4,embed_5,embed_6,embed_7,embed_8,embed_9,...,embed_54,embed_55,embed_56,embed_57,embed_58,embed_59,embed_60,embed_61,embed_62,embed_63
0,-0.133523,0.225222,0.314113,0.181442,-0.015446,-0.155780,-0.350083,0.169681,0.396281,-0.101996,...,0.158478,0.160546,-0.215638,-0.245130,0.198993,0.329484,0.378549,0.372550,0.048838,0.243953
1,-0.243253,0.261145,0.419577,0.186948,-0.027285,-0.072425,-0.307995,0.247811,0.419245,-0.167111,...,0.141347,0.148349,-0.156413,-0.294571,0.300025,0.505661,0.396157,0.280961,-0.019553,0.269522
2,-0.222710,0.249139,0.420708,0.239597,-0.043783,-0.123580,-0.331820,0.236480,0.421185,-0.136183,...,0.166336,0.153981,-0.201262,-0.314124,0.286887,0.507230,0.403490,0.317734,-0.025989,0.233260
3,-0.304725,0.213599,0.468104,0.300333,0.002506,-0.100246,-0.225437,0.174049,0.491740,-0.128044,...,0.135983,0.188145,-0.145956,-0.375446,0.291373,0.400195,0.440007,0.378852,-0.027192,0.137819
4,-0.176236,0.197553,0.407351,0.203850,-0.016887,-0.112404,-0.296417,0.251118,0.380544,-0.181026,...,0.161730,0.180895,-0.210305,-0.303275,0.245013,0.433772,0.369498,0.344984,-0.023950,0.215005


In [14]:
# Binary classification
import numpy as np
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.models import Model


print(processed_full_df['label'].unique())

binary_target = (processed_full_df['label'] != 0).astype(int).values
unique, counts = np.unique(binary_target, return_counts=True)
print("Classes and their counts:")
for cls, cnt in zip(unique, counts):
    print(f"Class {cls}: {cnt}")

[0 1]
Classes and their counts:
Class 0: 93000
Class 1: 164673


In [15]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from transformer import BasicTransformer
from classification import LastTokenClassificationHead
from sklearn.model_selection import train_test_split
import numpy as np  # Just in case

# Input data
X_embed = embedded_df.values                   # shape: (num_samples, embed_dim)
X_embed_seq = tf.expand_dims(X_embed, axis=1)  # shape: (batch_size, 1, embed_dim)

input_layer = Input(shape=(1, X_embed.shape[1]), name="record_input")

# Transformer block
transformer = BasicTransformer(
    n_layers=2,
    internal_size=128,
    n_heads=4,
    verbose=True  # See progress!
)

x = transformer.apply(input_layer, training=True)

# Classification head
classification_head = LastTokenClassificationHead()
x = classification_head.apply(x)

# MLP head
for i, layer_size in enumerate([128]):
    x = Dense(layer_size, activation="relu", name=f"classification_mlp_{i}_{layer_size}")(x)
    x = Dropout(0.1)(x)

# Output layer for binary classification
output_layer = Dense(1, activation="sigmoid", name="binary_output")(x)

# Build the model
model = Model(inputs=input_layer, outputs=output_layer)
model.summary()

# Compile the model
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"],
    jit_compile=True
)

# Prepare data
X_np = X_embed_seq.numpy() if hasattr(X_embed_seq, 'numpy') else np.array(X_embed_seq)

X_train, X_val, y_train, y_val = train_test_split(
    X_np, binary_target, test_size=0.2, random_state=42, stratify=binary_target
)

# Define early stopping callback
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

# Train the model
model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=32,
    callbacks=[early_stop]
)


[BasicTransformer] Applying Encoder Block 1/2
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[BasicTransformer] Applying Encoder Block 2/2
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[BasicTransformer] Final output shape: (None, 1, 64)


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ record_input (InputLayer)       │ (None, 1, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block_0_transformer_encoder     │ (None, 1, 64)          │       149,504 │
│ (TransformerEncoderBlock)       │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block_1_transformer_encoder     │ (None, 1, 64)          │       149,504 │
│ (TransformerEncoderBlock)       │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ slice_last (Lambda)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ classification_mlp_0_128        │ (None, 128)            │         8,320 │
│ (Dense)                         │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ binary_output (Dense)           │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 307,457 (1.17 MB)

 Trainable params: 307,457 (1.17 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
6442/6442 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8804 - loss: 0.2469[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
6442/6442 ━━━━━━━━━━━━━━━━━━━━ 44s 5ms/step - accuracy: 0.8804 - loss: 0.2469 - val_accuracy: 0.8921 - val_loss: 0.2142
Epoch 2/50
6442/6442 ━━━━━━━━━━━━━━━━━━━━ 25s 4ms/step - accuracy: 0.8912 - loss: 0.2163 - val_accuracy: 0.8924 - val_loss: 0.2131
Epoch 3/50
6442/6442 ━━━━━━━━━━━━━━━━━━━━ 25s 4ms/step - accuracy: 0.8954 - loss: 0.2072 - val_accuracy: 0.8933 - val_loss: 0.2155
Epoch 4/50
6442/6442 

In [16]:
# evaluate the model
results = model.evaluate(X_val, y_val)
print(f"Validation loss: {results[0]:.4f}, Validation accuracy: {results[1]:.4f}")


1611/1611 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8994 - loss: 0.1774
Validation loss: 0.1791, Validation accuracy: 0.8995


In [17]:
from sklearn.metrics import f1_score, confusion_matrix
import numpy as np

# Predict probabilities
y_pred_prob = model.predict(X_val)
y_pred = (y_pred_prob >= 0.5).astype(int).flatten()

# F1
f1 = f1_score(y_val, y_pred)
print(f"F1 Score: {f1:.4f}")

# Confusion matrix
cm = confusion_matrix(y_val, y_pred, labels=[0, 1])
print("Confusion Matrix:\n", cm)

if cm.shape == (2, 2):
    tn, fp, fn, tp = cm.ravel()
    detection_rate = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    false_alarm_rate = fp / (fp + tn) if (fp + tn) > 0 else 0.0
    print(f"Detection Rate (Recall): {detection_rate:.4f}")
    print(f"False Alarm Rate: {false_alarm_rate:.4f}")
else:
    print("Confusion matrix does not contain both classes!")
    print("Check your labels: y_val:", np.unique(y_val), " y_pred:", np.unique(y_pred))


[EncoderBlock] Input shape: (32, 1, 64)
[EncoderBlock] Output shape: (32, 1, 64)
[EncoderBlock] Input shape: (32, 1, 64)
[EncoderBlock] Output shape: (32, 1, 64)
1595/1611 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
1611/1611 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step
F1 Score: 0.9260
Confusion Matrix:
 [[13935  4665]
 [  515 32420]]
Detection Rate (Recall): 0.9844
False Alarm Rate: 0.2508
